## Dog Breed Classification Using MobileNetv2 (CNN)

### Importing all the libraries

In [2]:
import numpy as np

import tensorflow as tf

from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img  
from keras.models import Sequential  
from keras.layers import Dropout, Flatten, Dense  
from keras import applications  
from keras.utils.np_utils import to_categorical

import matplotlib.pyplot as plt  
import math

### Check for tensorflow version

In [3]:
tf.__version__

'2.4.1'

### Perimeters

In [4]:
img_width, img_height = 224, 224  

weights_path = 'mobilenet_model.h5'  

epochs = 10
batch_size = 64

### MobileNet model

In [5]:
model = applications.mobilenet.MobileNet(include_top=False, weights='imagenet')

### Training Set Data Preprocessing

In [6]:
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

train_set = train_datagen.flow_from_directory(  
                                   'dataset/training_set/cat_and_dog_breeds_tg',  
                                   target_size=(img_width, img_height),  
                                   batch_size=batch_size,  
                                   class_mode=None,  
                                   shuffle=False)

nb_train_samples = len(train_set.filenames) 
num_classes = len(train_set.class_indices) 

Found 30909 images belonging to 164 classes.


### MobileNetV2 Bottleneck Feature (Training Set)

In [7]:
predict_size_train = int(math.ceil(nb_train_samples / batch_size))  
   
bottleneck_features_train = model.predict(train_set, predict_size_train)  
   
np.save('bottleneck_features_train_mob.npy', bottleneck_features_train)

### Test Set Preprocessing Images

In [8]:
test_datagen = ImageDataGenerator(rescale = 1./255)
test_set = test_datagen.flow_from_directory('dataset/test_set/cat_and_dog_breeds_tt',
                                            target_size=(img_width, img_height),
                                            batch_size = batch_size,
                                            class_mode = None)

nb_test_samples = len(test_set.filenames)  

Found 30909 images belonging to 164 classes.


### MobileNetV2 Bottleneck Feature (Test Set)

In [9]:
predict_size_test = int(math.ceil(nb_test_samples / batch_size))  
   
bottleneck_features_test = model.predict(test_set, predict_size_test)  
   
np.save('bottleneck_features_test_mob.npy', bottleneck_features_test)

### Training Data to Categorial

In [10]:
training_datagen_top = ImageDataGenerator(rescale = 1./255)
training_set_top = training_datagen_top.flow_from_directory(  
         'dataset/training_set/cat_and_dog_breeds_tg',  
         target_size=(img_width, img_height),  
         batch_size=batch_size,  
         class_mode='categorical',  
         shuffle=False) 

nb_train_samples = len(training_set_top.filenames)  
num_classes = len(training_set_top.class_indices)  

Found 30909 images belonging to 164 classes.


### Uploading Bottleneck Features (Training Data)

In [11]:
train_data = np.load('bottleneck_features_train_mob.npy')  

train_labels = training_set_top.classes  
   
train_labels = to_categorical(train_labels, num_classes=num_classes)

### Uploading Bottlneck Feature (Test Data)

In [12]:
test_data = np.load('bottleneck_features_test_mob.npy')  

test_labels = test_set.classes  

test_labels = to_categorical(test_labels, num_classes=num_classes)

### CNN and MobileNet model

In [18]:
model = Sequential()  
model.add(Flatten(input_shape=train_data.shape[1:]))  
model.add(Dense(256, activation='relu'))  
model.add(Dropout(0.5))  
model.add(Dense(num_classes, activation='softmax'))  
   
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])  
   
history = model.fit(train_data, 
                    train_labels, 
                    epochs=epochs, 
                    batch_size=batch_size,
                    validation_data=(test_data, test_labels))  
   
model.save_weights(weights_path)  
   
(eval_loss, eval_accuracy) = model.evaluate(  
     test_data, test_labels, batch_size=batch_size, verbose=1)

print("[INFO] accuracy: {:.2f}%".format(eval_accuracy * 100))  
print("[INFO] Loss: {}".format(eval_loss))

Epoch 1/10
483/483 [==============================] - 125s 255ms/step - loss: 6.5578 - accuracy: 0.0076 - val_loss: 5.0929 - val_accuracy: 0.0091
Epoch 2/10
483/483 [==============================] - 116s 239ms/step - loss: 5.1246 - accuracy: 0.0102 - val_loss: 5.1141 - val_accuracy: 0.0091
Epoch 3/10
483/483 [==============================] - 111s 228ms/step - loss: 5.0919 - accuracy: 0.0127 - val_loss: 5.1314 - val_accuracy: 0.0090
Epoch 4/10
483/483 [==============================] - 112s 231ms/step - loss: 5.0645 - accuracy: 0.0204 - val_loss: 5.3823 - val_accuracy: 0.0089
Epoch 5/10
483/483 [==============================] - 108s 222ms/step - loss: 5.0284 - accuracy: 0.0249 - val_loss: 5.8699 - val_accuracy: 0.0090
Epoch 6/10
483/483 [==============================] - 99s 204ms/step - loss: 5.0142 - accuracy: 0.0315 - val_loss: 5.8415 - val_accuracy: 0.0087
Epoch 7/10
483/483 [==============================] - 99s 204ms/step - loss: 4.9684 - accuracy: 0.0398 - val_loss: 6.0642 - v

In [ ]:
# summarize history for accuracy  
   
plt.subplot(211)  
plt.plot(history.history['acc'])  
plt.plot(history.history['val_acc'])  
plt.title('MobileNet model accuracy')  
plt.ylabel('accuracy')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
  
# summarize history for loss  
   
plt.subplot(212)  
plt.plot(history.history['loss'])  
plt.plot(history.history['val_loss'])  
plt.title('model loss')  
plt.ylabel('loss')  
plt.xlabel('epoch')  
plt.legend(['train', 'test'], loc='upper left')  
plt.show()